In [9]:
import json
import pandas as pd
import re
import numpy as np

from keras.models import Sequential
from keras.layers import Dense, LSTM, Embedding

from keras.preprocessing import sequence
from keras.utils import np_utils

import os
import shutil
import urllib3
import zipfile
import pandas as pd
import json
import ast

In [10]:
f = open("[라벨]경상도_학습데이터_1/DKCI20000001.json", encoding="UTF-8")
raw_data = json.loads(f.read())

In [11]:
# 사투리, 표준어 사전 생성 함수
def make_dict(raw_data):

    dialect_word = []
    standard_word = [] 
    
    
    for i in range(len(raw_data["utterance"])):
        eojeol = raw_data["utterance"][i]["eojeolList"]
            
        for k in range(len(eojeol)):
            if eojeol[k]["isDialect"] == True:
                if k not in dialect_word:
                    dialect_word.append(eojeol[k]['eojeol'])
                    standard_word.append(eojeol[k]['standard'])

    word_dictionary = pd.DataFrame([dialect_word, standard_word])
    word_dictionary = word_dictionary.transpose()
    word_dictionary.columns = ['dialect', 'standard']
    dialect_dictionary = word_dictionary['dialect']
    standard_dictionary = word_dictionary['standard']
    dialect_dict = dialect_dictionary.to_dict()
    standard_dict = standard_dictionary.to_dict()

    index_to_dialect = {v:k for k,v in dialect_dict.items()}
    index_to_standard = {v:k for k,v in standard_dict.items()}

    dialect_to_index = {v:k for k,v in index_to_dialect.items()}
    standard_to_index = {v:k for k,v in index_to_standard.items()}

    print(len(index_to_dialect))
    print(len(dialect_to_index))

    print(len(index_to_standard))
    print(len(standard_to_index))
    

    return dialect_to_index,standard_to_index,index_to_dialect,index_to_standard
   

In [12]:
index_to_dialect,index_to_standard,dialect_to_index,standard_to_index = make_dict(raw_data)

236
236
223
223


In [13]:
# 사투리 표준어 쌍 생성 함수
def make_pair(raw_data):
    filter = re.compile('[^가-힣+]')

    dialect_form = []
    standard_form = []

    for i in range(len(raw_data["utterance"])):
        sentence = raw_data["utterance"][i]
        
        for j in range(len(sentence["eojeolList"])):
            #print(sentence["eojeolList"][0]['eojeol'])
            if sentence["eojeolList"][j]['isDialect'] == True:
                try : 

                    dialect = sentence["eojeolList"][j]["eojeol"]
                    standard = sentence["eojeolList"][j]["standard"]
                    
                    dialect = filter.sub('', dialect)
                    standard = filter.sub('', standard)
                    
                    dialect = dialect_to_index[dialect]
                    standard = standard_to_index[standard]

                    dialect_form.append(dialect)
                    standard_form.append(standard)
                except:
                    pass

    result = pd.DataFrame(data = {"dialect_words":dialect_form, "standard_words": standard_form})


    return result
               

In [14]:
pairs = make_pair(raw_data)

In [15]:
# 사투리 사전에 존재하는지 확인하는 함수

def exist_in_dialect(x):

    if x in dialect_to_index:
        return True
    else:
        return False


In [16]:
exist_in_dialect("자가격리")

False

In [186]:
xtrain = np_utils.to_categorical(pairs["dialect_words"])
ytrain = np_utils.to_categorical(pairs["standard_words"])
len(xtrain[0])

416

In [187]:
xtrain.shape

(398, 416)

In [188]:
translate_model = Sequential()
translate_model.add(Dense(256, input_dim = 416, activation="relu"))
translate_model.add(Dense(416, activation="softmax"))

In [189]:
translate_model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [190]:
translate_model.fit(xtrain, ytrain, batch_size=100, epochs=100)

Epoch 1/100
4/4 [==============================] - 1s 5ms/step - loss: 6.0195 - accuracy: 0.0025
Epoch 2/100
4/4 [==============================] - 0s 5ms/step - loss: 5.9636 - accuracy: 0.1985
Epoch 3/100
4/4 [==============================] - 0s 4ms/step - loss: 5.9074 - accuracy: 0.3392
Epoch 4/100
4/4 [==============================] - 0s 3ms/step - loss: 5.8494 - accuracy: 0.4322
Epoch 5/100
4/4 [==============================] - 0s 5ms/step - loss: 5.7853 - accuracy: 0.4673
Epoch 6/100
4/4 [==============================] - 0s 5ms/step - loss: 5.7161 - accuracy: 0.4774
Epoch 7/100
4/4 [==============================] - 0s 5ms/step - loss: 5.6371 - accuracy: 0.4724
Epoch 8/100
4/4 [==============================] - 0s 8ms/step - loss: 5.5484 - accuracy: 0.4749
Epoch 9/100
4/4 [==============================] - 0s 5ms/step - loss: 5.4495 - accuracy: 0.4623
Epoch 10/100
4/4 [==============================] - 0s 3ms/step - loss: 5.3387 - accuracy: 0.4523
Epoch 11/100
4/4 [===========

In [193]:
# 사투리 단어 번역 함수
def translate(word):
    for i in range(len(sentence)):
        if exist_in_dialect(sentence[i]) == True:
            print(dialect_to_index[sentence[i]])
            index = dialect_to_index[sentence[i]]
            one_hot = np_utils.to_categorical(index,len(xtrain[0])) #인덱스 배열의크기 지정 필요 size_of_dialect
            pred_index = np.argmax(translate_model.predict(np.array([one_hot])))
            res = res + " " + index_to_standard[pred_index]

        else:
            res = res + " " + sentence[i]

    return res


In [194]:
translate("자가격리 이주 했다 아이가")

357
407


' 자가격리 이주 했지 않니'

In [24]:
exist_in_dialect("인덱스")

False

In [25]:
pos_dict = {}

In [26]:
def make_eojeol():
    res = 0

    return res

In [27]:
eojeol_dict = make_eojeol()


In [28]:
def eojeol_dependence():
    res = 0
    return res

In [29]:
df_eojeol_dependence = eojeol_dependence()

In [198]:
def predict_synonym(sentence):
    res = ""
    sentence_sep = sentence.split()
    print(sentence_sep)
    for i in range(len(sentence_sep)):
        print(i)
        if exist_in_dialect(sentence_sep[i]) == True: #사투리 사전에 있으면
            if sentence_sep[i] in eojeol_dict.keys(): #해당 단어가 전체 사전에 있으면

                pos, _ = get_mods(sentence, i) #연관된 단어의 품사 받아옴
                pos_idx = pos_dict[pos] #품사의 인덱스
                word = sentence_sep[i] 
                word0_idx = eojeol_dict[word+"0"] #단어0의 인덱스
                word_idx = eojeol_dict[word] #단어의 인덱스

                #one_hot = np_utils.to_categorical(pos_idx,len(xtrain[0])) #인덱스 배열의크기 지정 필요 size_of_dialect
                percentage = tmp_model.predict(np.array([one_hot]))
                res_idx = max(percentage[word0_idx], percentage[word_idx]) # 둘 중 더 큰 확률값
                
                one_hot = np_utils.to_categorical(res_idx,len(xtrain[0])) #인덱스 배열의크기 지정 필요 size_of_dialect
                pred_index = np.argmax(translate_model.predict(np.array([one_hot])))


                #pred_index = np.argmax(translate_model.predict(np.array([one_hot])))
                

                print("yes")
                #model.predict()
                res = res + " " + word+"0"
            else:
                word = sentence_sep[i] #단어의 인덱스
                word0_idx = eojeol_dict[word+"0"] #단어0의 인덱스
                one_hot = np_utils.to_categorical(index,len(xtrain[0])) #인덱스 배열의크기 지정 필요 size_of_dialect
                pred_index = np.argmax(translate_model.predict(np.array([one_hot])))
        else:
            res = res + " " + sentence_sep[i]
            
    
    return res

In [201]:
max(1,2)

2

In [199]:
pos_dict = {'VP_MOD':0, "VP":1}
eojeol_dict = {"아이가0":0, "아이가":1, "있지":2, "않니":3}

In [200]:
tmp = predict_synonym("착한 아이가 있다 아이가")
tmp

['착한', '아이가', '있다', '아이가']
0
1
아이가
yes
2
3
아이가
yes


' 착한 아이가0 있다 아이가0'

In [161]:
def get_mods(sentence, num):
    openApiURL = "http://aiopen.etri.re.kr:8000/WiseNLU"

    accessKey = "aba3a45d-3318-4061-8a5d-799c8521b082"
    analysisCode = "dparse"

    text = sentence
    
    requestJson = {
        "access_key": accessKey,
        "argument": {
            "text": text,
            "analysis_code": analysisCode
        }
    }

    http = urllib3.PoolManager()

    response = http.request(
        "POST",
        openApiURL,
        headers={"Content-Type": "application/json; charset=UTF-8"},
        body=json.dumps(requestJson)
    )

    tmp = ast.literal_eval(response.data.decode('utf-8'))
    dict_with_mods = tmp["return_object"]["sentence"][0]["dependency"]

    #print(dict_with_mods[num]["mod"][0])
    id=int(dict_with_mods[num]["mod"][0])
    pos = dict_with_mods[id]["label"]
    
    return pos, dict_with_mods[id]["text"]

In [162]:
get_mods("착한 아이가 있다 아이가", 1)

('VP_MOD', '착한')

In [35]:
index = dialect_to_index["아이가"]
one_hot = np_utils.to_categorical(index,len(xtrain[0])) #인덱스 배열의크기 지정 필요 size_of_dialect
pred_index = np.argmax(model.predict(np.array([one_hot])))

In [38]:
model.predict(np.array([one_hot]))

array([[3.41646337e-05, 7.66829089e-07, 5.57304702e-07, 5.62017021e-07,
        4.90441664e-07, 1.53379151e-05, 8.81171729e-07, 7.29771898e-07,
        5.64388927e-07, 1.70652456e-05, 7.40889561e-07, 6.73411364e-07,
        2.89047039e-05, 3.17146187e-05, 8.27689462e-07, 6.84502197e-07,
        1.99444639e-05, 8.16351530e-07, 5.48711569e-07, 3.17239974e-05,
        7.18649687e-07, 1.86942889e-05, 6.73435750e-07, 7.89718229e-07,
        5.94670666e-07, 8.90942488e-07, 4.44952150e-07, 4.12420668e-05,
        5.75113404e-07, 5.61686420e-07, 1.00661180e-06, 3.12426491e-05,
        4.65670419e-05, 1.03706725e-05, 7.65277377e-07, 6.62722016e-07,
        2.31959639e-05, 5.93304947e-07, 3.00328702e-05, 5.11552535e-05,
        1.05850932e-05, 2.69867633e-05, 1.27453832e-05, 8.27230963e-07,
        1.40969460e-05, 7.86846499e-07, 8.47686181e-07, 6.74459613e-07,
        1.47911787e-05, 6.35333379e-07, 5.58367617e-07, 1.10529973e-05,
        7.88745012e-07, 6.93989080e-07, 7.10668814e-07, 3.177898